### Introduction

The following datasets are used in this demonstration:
- 2011 Census data - polulation and household estimates, available [here](https://www.ons.gov.uk/census/2011census/2011censusdata/2011censusdatacatalogue/populationandhouseholdestimates).
- ONS - House price to residence-based earnings ratio, available [here](https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/ratioofhousepricetoresidencebasedearningslowerquartileandmedian)

### Table of contents

1. [Population and household estimates (univariate)](#section1)

    1.1 [Gender ratio](#section1_1)

    1.2 [Gender by region](#section1_2)

    1.3 [Population by outward postcode using choropleth map](#section1_3)

    1.4 [Population by postcode parent area using choropleth map](#section1_4)

    1.5 [Male/Female ratio by postcode parent area using choropleth map](#section1_5)

2. [House prices and earnings (multivariate)](#section2)

### Preparation
Please run this code cell to install required packages. If this does not work, please install these packages manually in your virtual environment.

In [ ]:
# Install packages using jupyter notebook's built-in %pip function
%pip install pandas plotly geojson shapely openpyxl scipy nltk

Import packages to be used and replace default plotting backend.

In [ ]:
# Import required packages
import pandas as pd
import numpy as np
import json
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import shapely.geometry
from shapely.ops import unary_union
import geojson
import nltk

# Set default pandas plotting backend to Plotly
pd.options.plotting.backend = "plotly"

<a id='section1'></a>
### 1. Population and household estimates

In [ ]:
# Load data
df1 = pd.read_csv("../data/ons/population_household.csv")

# Split postcode into outward and inward, e.g. LE3 9QP => LE3 and 9QP
df1["postcode_out"] = df1["Postcode"].apply(lambda x: x[0:4].strip())
df1["postcode_in"] = df1["Postcode"].apply(lambda x: x[4:].strip())

# Keep only non-numeric part in outward as the parent region
df1["postcode_region"] = df1["postcode_out"].apply(lambda x: re.split("\d+", x)[0])

# Remove column [Postcode] to save memory
df1 = df1.drop(["Postcode"], axis=1)

<a id='section1_1'></a>
#### 1.1 Gender ratio

In [ ]:
# Count total number of males and females
df_gender = pd.DataFrame({"Gender": ["Male", "Female"], "Count": [df1["Males"].sum(), df1["Females"].sum()]})

# Generate pie chart
gender_pie = px.pie(df_gender, values="Count", names="Gender", title="National gender ratio (England and Wales)")
gender_pie.show()

# Remove variables to save memory
del df_gender, gender_pie

<a id='section1_2'></a>
#### 1.2 Population and gender ratio by region

In [ ]:
# Sum up all numbers within same postcode region, descending order by column [Total]
df_temp = df1.groupby(["postcode_region"]).sum().sort_values(by=['Total'], ascending=False).reset_index()

# Generate plots side by side
fig_temp = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=False, vertical_spacing=0.001)

# Add plot 1: bar chart for top 10 population regions
fig_temp.append_trace(go.Bar(
    x=df_temp.head(10)["Total"],
    y=df_temp.head(10)["postcode_region"],
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='Population',
    orientation='h',
), 1, 1)

# Add plot 2: scatter chart for gender ratio in top 10 population regions
fig_temp.append_trace(go.Scatter(
    x=df_temp.head(10)["Males"]/df_temp.head(10)["Females"], y=df_temp.head(10)["postcode_region"],
    mode='lines+markers',
    line_color='rgb(128, 0, 128)',
    name='M/F ratio',
), 1, 2)

# Update layout changes on plot title, axes, color and legends
fig_temp.update_layout(
    title='Male/Female ratio in top 10 population regions',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top',
        dtick=25000,
    ),
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

# Add annotations to the plot
annotations = []

# Adding labels to charts
for a_mf, a_pop, x_pcr in zip(np.round(df_temp.head(10)["Males"]/df_temp.head(10)["Females"], decimals=2), df_temp.head(10)["Total"], df_temp.head(10)["postcode_region"]):
    # Add label to M/F ratio scatter plot
    annotations.append(dict(xref='x2', yref='y2',
                            y=x_pcr, x=a_mf,
                            text=a_mf,
                            xshift=50,
                            showarrow=False))
    # Add label to population bar chart
    annotations.append(dict(xref='x1', yref='y1',
                            y=x_pcr, x=a_pop,
                            text=f"{np.round(a_pop/1000000, 3)}M",
                            xshift=25,
                            showarrow=False))

fig_temp.update_layout(annotations=annotations)
fig_temp.show()

In [ ]:
# Remove variables to save memory
del df_temp, annotations, fig_temp

<a id='section1_3'></a>
#### 1.3 Population by outward postcode using choropleth map
Concat individual postcode geojson mapping into single variable `geojson_uk`.

In [ ]:
# Create parent geojson collection object
geojson_uk_granulated = dict(type="FeatureCollection", features=list())
# Load all geojson files
path_geojson = Path("../data/geojson")
for f_geojson in list(path_geojson.glob("*.geojson")):
    with open(f_geojson) as f:
        geojson_data = json.load(f)
        for feature in geojson_data["features"]:
            # Add feature id using properties.name, which is the outward postcode
            feature["id"] = feature["properties"]["name"]
            # Add feature to parent geojson collection
            geojson_uk_granulated["features"].append(feature)

Plot population with postcode area. Please note that as the opensource geographical data I used in this demonstration comes from Wikipedia, it does not cover all England regions. This leads to white areas on the map.

In [ ]:
# Sum up all numbers within same postcode outward area
df_temp = df1.groupby(["postcode_out"]).sum().reset_index()

# Get max population number among areas
p_max = df_temp["Total"].max()

# Plot figure
fig = px.choropleth_mapbox(df1.groupby(["postcode_out"]).sum().reset_index(), geojson=geojson_uk_granulated,
                           locations='postcode_out', color='Total',
                           color_continuous_scale="jet",
                           range_color=(0, p_max),
                           mapbox_style="carto-positron",
                           zoom=4.5, center={"lat": 52.5, "lon": -1.6},
                           opacity=0.5
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

# Remove variables to save memory
del df_temp, p_max, fig

<a id='section1_4'></a>
#### 1.4 Population by postcode parent area using choropleth map
In the above plot, regions are probably too granulated, so you would not be able to see an obvious trend. Let's merge them into parent regions.

In [ ]:
# Create parent geojson collection object
geojson_uk_regional = dict(type="FeatureCollection", features=list())

# Load all geojson files
path_geojson = Path("../data/geojson")
for f_geojson in list(path_geojson.glob("*.geojson")):
    with open(f_geojson) as f:
        geojson_data = json.load(f)
        # Merge granulated geometry areas into parent region
        merged = unary_union(
            list(
                map(
                    (lambda x: shapely.geometry.asShape(x["geometry"])),
                    geojson_data["features"])
            )
        )
        # Create new geojson feature from merged geometry
        geojson_merged = geojson.Feature(geometry=merged,
                                         properties={"name": f_geojson.stem},
                                         id=f_geojson.stem)
        # Add feature to parent geojson collection
        geojson_uk_regional["features"].append(geojson_merged)

Now we have merged

In [ ]:
# Sum up all numbers within same postcode region
df_temp = df1.groupby(["postcode_region"]).sum().reset_index()

# Get max population number among regions
p_max = df_temp["Total"].max()

# Plot figure
fig = px.choropleth_mapbox(df_temp,
                           geojson=geojson_uk_regional,
                           locations='postcode_region',
                           color='Total',
                           color_continuous_scale="jet",
                           range_color=(0, p_max),
                           mapbox_style="carto-positron",
                           zoom=4.5, center={"lat": 52.5, "lon": -1.6},
                           opacity=0.5
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

# Remove variables to save memory
del df_temp, p_max, fig

In [ ]:
# Remove dataframe used for section 1
del df1

<a id='section2'></a>
### 2. House prices and earnings

Define a reusable function to read from the source file and process into a DataFrame in required format.

In [ ]:
def read_df2(source, sheet, skip_top, column_name):
    """
    :param source: str or path object, where the source excel file located
    :param sheet: str, name of the sheet in excel file to be read
    :param skip_top: int, how many rows at the top of the sheet to be skipped
    :param column_name: str, new column name to be given to the variable
    :return: DataFrame
    """

    def transpose_df2(r, c_name):
        """
        :param r: Series, a row from the DataFrame
        :param c_name: str, new name to be given to transposed column
        :return: DataFrame
        """
        #%% row -> column -> new DataFrame
        new_df = r[year_columns].T.to_frame().reset_index(drop=True)
        #%% Rename the column
        new_df.columns = [c_name]
        #%% Add column [Region] and populate the whole column with value from cell [Region] in the row
        new_df["Region"] = r["Region"].strip()
        #%% Add column [Year] and populate the column from 2002 to 2020
        new_df["Year"] = range(2002, 2021)
        return new_df

    #%% Read target sheet from source file into DataFrame -> drop column [Code] -> drop row if any cell in column [Name] is empty
    temp = pd.read_excel(source, sheet_name=sheet, skiprows=skip_top).drop(columns=["Code"]).dropna(subset=["Name"])
    # As the source file contains empty columns, drop them
    empty_cols = [col for col in temp.columns if temp[col].isnull().all()]
    temp.drop(empty_cols, axis=1, inplace=True)
    # Columns names are inconsistent in different sheets, replace them with standard year based column names
    year_columns = [str(x) for x in range(2002, 2021)]
    temp.columns = ["Region"] + year_columns
    temp.loc[0][year_columns].T.reset_index(drop=True)
    new_temp = pd.DataFrame(columns=["Region", "Year", column_name])
    # Transpose and concat each row into the final DataFrame
    for _, row in temp.iterrows():
        new_temp = pd.concat([new_temp, transpose_df2(row, column_name)])
    # Convert numeric column data type to integer
    new_temp = new_temp.astype({column_name: 'int32'})
    return new_temp

Read dataset into DataFrame

In [ ]:
# Read sheet [1a] for the median house price by country and region, England and Wales, 2002 - 2020
df2_1a = read_df2("../data/ons/house_price_earning.xlsx", "1a", 6, "Median house price")
# Read sheet [1b] for the median gross annual earnings by country and region, England and Wales, 2002 - 2020
df2_1b = read_df2("../data/ons/house_price_earning.xlsx", "1b", 6, "Median earn")
# Read sheet [2a] for the lower quartile house price by country and region, England and Wales, 2002 - 2020
df2_2a = read_df2("../data/ons/house_price_earning.xlsx", "2a", 6, "Lower house price")
# Read sheet [1b] for the lower quartile gross annual earnings by country and region, England and Wales, 2002 - 2020
df2_2b = read_df2("../data/ons/house_price_earning.xlsx", "2b", 6, "Lower earn")
# Merge the four DataFrame above into new DataFrame
df2_final = pd.merge(df2_1a, df2_1b, on=["Region", "Year"])
df2_final = pd.merge(df2_final, df2_2a, on=["Region", "Year"])
df2_final = pd.merge(df2_final, df2_2b, on=["Region", "Year"])
# Show top 5 rows of processed DataFrame
df2_final.head(5)
# Delete variables to save memory
del df2_1a, df2_1b, df2_2a, df2_2b

<a id='section2_1'></a>
#### 2.1 Correlation between median house price and median household earning

- Scatter plot and trendlines show the correlations between median house price and median earn.
- Top box plot shows the distribution of median house price.
- Right rug box shows the distribution of median earn.

In [ ]:
df2_fig1 = px.scatter(df2_final[df2_final.Region.isin(["England", "Wales", "London"])], x="Median house price", y="Median earn", color="Region",
                 marginal_x="box", marginal_y="rug", trendline="ols")
df2_fig1.show()

In [ ]:
df2_temp = pd.DataFrame(columns=["Region", "Year", "Variable", "Value"])
for _, row in df2_final.iterrows():
    df2_temp = df2_temp.append([{"Region": row["Region"], "Year": row["Year"], "Variable": "Median house price", "Value": row["Median house price"]},
                              {"Region": row["Region"], "Year": row["Year"], "Variable": "Median earn", "Value": row["Median earn"]}, {"Region": row["Region"], "Year": row["Year"], "Variable": "Lower house price", "Value": row["Lower house price"]}, {"Region": row["Region"], "Year": row["Year"], "Variable": "Lower earn", "Value": row["Lower earn"]}], ignore_index=True)

df2_fig2 = px.bar(df2_temp[df2_temp["Variable"].isin(["Median house price", "Lower house price"])], x="Region", y="Value", text="Value", color="Variable", barmode="group", animation_frame="Year", animation_group="Region", range_y=[0,500000])
df2_fig2.update_yaxes(title_text="Price", secondary_y=False)
df2_fig2.update_yaxes(title_text="Earn", secondary_y=True)
df2_fig2.show()

del df2_final, df2_temp, df2_fig1, df2_fig2

<a id='section3'></a>
### 3. Text based analysis and visualisation